# CLIMATE DATA DE BILT 

Bu notebook, Hollanda'daki 1800-2014 yılları arasındaki iklim normallerinin kaydedildiği bir veri seti kullanılarak iklim verilerinin birbiriyle ilişkisini analiz etmek ve bir tahmin modeli gelistirmek amacıyla **veri işleme** ve **Python uygulamaları** üzerine bir çalışma içermektedir.

##### Bu notebook spesifik olarak Heating Degree Days Tahmin Modeli kısmına yoğunlaşmaktadır.

In [3]:
import pandas as pd
import requests
import numpy as np

In [5]:
import matplotlib.pyplot as plt

In [7]:
# Data Preprocessing notebookunda temizlenmiş veri setinin bulunduğu Excel dosyasını içe aktarma
file_path = "ClimateDeBilt_Preprocessed.xlsx"
df = pd.read_excel(file_path)

In [9]:
#Data setimin içindeki veri tiplerini kontrol ediyorum 
df.dtypes


Year                         int64
YearlyAvgTemp              float64
WinterAvgTemp              float64
WinterAvgMinTemp           float64
SummerAvgTemp              float64
SummerAvgMaxTemp           float64
SunlessDays                  int64
DaysWithPrecipitation        int64
DaysWithFog                  int64
IceDays                      int64
FrostDays                    int64
SnowDays                     int64
HoursOfSunshine              int64
SummeryDays                  int64
TropicalDays                 int64
DryDays                      int64
RelativeHumidity             int64
QuantityOfPrecipitation      int64
Evaporation                  int64
HeatingDegreeDays            int64
dtype: object

In [11]:
# Veri çerçevesindeki eksik veri olup olmadığını kontrol ediyorum
missing_data = df.isnull().sum()

# Sonuçları görüntülüyorum
print(missing_data)


Year                       0
YearlyAvgTemp              0
WinterAvgTemp              0
WinterAvgMinTemp           0
SummerAvgTemp              0
SummerAvgMaxTemp           0
SunlessDays                0
DaysWithPrecipitation      0
DaysWithFog                0
IceDays                    0
FrostDays                  0
SnowDays                   0
HoursOfSunshine            0
SummeryDays                0
TropicalDays               0
DryDays                    0
RelativeHumidity           0
QuantityOfPrecipitation    0
Evaporation                0
HeatingDegreeDays          0
dtype: int64


In [13]:
print(df.shape)  # Veri setinin boyutu

(35, 20)


##### Her şey doğru. Model Geliştirme kısmına geçebilirim.

## Model Training 1.0 : Random Split of Train $ Test Sets

### Veriyi Train ve Test setlerine Ayırma

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Y = Target (Heating Degree Days) ve X = regressorlerin yazilimi (Ilk basta tum degiskenleri kullanarak bir model gelistirecegim)
X = df.drop("HeatingDegreeDays", axis=1)
y = df["HeatingDegreeDays"]

# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model oluşturma ve eğitme
model = LinearRegression()
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)

# Performans metrikleri
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")


Mean Squared Error: 1457.3843723941447
R²: 0.9159785648513125


### Modelin overfitting problemi olup olmadığını kontrol etmemiz gerekiyor.

In [21]:
# Eğitim seti performansı
y_train_pred = model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Test seti performansı
y_test_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Sonuçları yazdırma
print(f"Eğitim Seti MSE: {train_mse}, R²: {train_r2}")
print(f"Test Seti MSE: {test_mse}, R²: {test_r2}")


Eğitim Seti MSE: 248.76139995481503, R²: 0.9969232426570068
Test Seti MSE: 1457.3843723941447, R²: 0.9159785648513125


#### Yorum: 

Modelin eğitim ve test setlerindeki MSE değerlerinin birbirinden çok uzak değerler olduğunu görüyoruz. Buradan anlıyoruz ki model eğitim setine uyum gösterirken test setinde performansı düşmekte. Overfittin Yani "aşırı öğrenme" problemi ile karşı karşıyayız. Sonuç: Veri setindeki fazla sayıda değişken, modelin eğitim setine fazla adapte olmasını yani ayrıntılarda kaybolduğunu göstermekte.

#### Öneri: 

Heating Degree Days değişkeni zamana bağlı olarak azalan bir değer olduğu için, model eğitimi esnasında veri setinin %70'ini modeli eğitmek bunun için de 1981-2003 yılları arasındaki verileri kullanmak, %30'unu modeli test etmek bunun için de 2004-2014 yıllarındaki verileri kullanabiliriz. 

## Model Training 2.0 : Year-based Split of Train $ Test Sets

### Veriyi Train ve Test setlerine Ayırma

In [27]:

# Eğitim ve test setlerini yıllara göre ayırma
train_data = df[df['Year'] < 2004]  # 1981-2004 yılları
test_data = df[df['Year'] >= 2004]  # 2004-2014 yılları

# Eğitim seti
X_train = train_data.drop('HeatingDegreeDays', axis=1)  # Eğitim setinde HDD dışındaki tüm değişkenler
y_train = train_data['HeatingDegreeDays']  # Eğitim setindeki HDD değeri

# Test seti
X_test = test_data.drop('HeatingDegreeDays', axis=1)  # Test setinde HDD dışındaki tüm değişkenler
y_test = test_data['HeatingDegreeDays']  # Test setindeki HDD değeri

# Modeli oluşturma ve eğitme
model = LinearRegression()
model.fit(X_train, y_train)

# Eğitim seti ile tahmin yapma
y_train_pred = model.predict(X_train)

# Test seti ile tahmin yapma
y_test_pred = model.predict(X_test)

# Eğitim ve test seti performans değerlendirmesi
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Sonuçları yazdırma
print(f"Eğitim Seti MSE: {train_mse}, R²: {train_r2}")
print(f"Test Seti MSE: {test_mse}, R²: {test_r2}")

Eğitim Seti MSE: 124.10053603777978, R²: 0.9980187053622535
Test Seti MSE: 26552.16283381573, R²: 0.5563713798042971


#### Yorum: 

Yeni modelin test setindeki R-score değeri eğitim setinden azımsanamayacak derecede daha düşük. Yeni modelin test setindeki MSE değerinin ise daha da arttığını görmekteyiz.

#### Model 1.0 ve Model 2.0 Karşılaştırılması

Model 1.0 : Random Split Sets
Model 2.0 : Year-based Split Sets

In [31]:

data = {
    "Model": ["Random Split", "Year-based Split"], 
    "Train MSE": [248.76, 124.10],    # Model 1 ve Model 2 için Train MSE değerleri
    "Test MSE": [1457.38, 26552.16],     # Model 1 ve Model 2 için Test MSE değerleri
    "Train R²": [0.99, 0.99],         # Model 1 ve Model 2 için Train R² değerleri
    "Test R²": [0.91, 0.55]           # Model 1 ve Model 2 için Test R² değerleri
}

# Verileri bir DataFrame'e dönüştürme
comparison_df = pd.DataFrame(data)

# Tabloyu yazdırma
print(comparison_df)


              Model  Train MSE  Test MSE  Train R²  Test R²
0      Random Split     248.76   1457.38      0.99     0.91
1  Year-based Split     124.10  26552.16      0.99     0.55


#### Öneri: 

R-score ve MSE değerlerinin test ve eğitim setlerinde farklı olmalarının sebebi, Multicollinearity yani bağımsız değişkenlerin (predictor) arasında yüksek lineer ilişki olmaası yani birbiriyle çok güçlü ilişkide olmaları anlamına gelebilir. Bu sebeple Multicollinearity Detection yapmak gerekmektedir.

Multicollinearity Tespiti için kullanılabilecek yöntemler:

1. Korelasyon matrixi kullanılarak r değerlerinin 0.8'den ve/veya -0.8'den fazla olan değişkenler tespit edilebilir.
    Korelasyon matrixi adımında Heating Degree Days ile arasında yüksek pozitif ve yüksek negatif ilişki olan değişkenler:
       Frost Days (r=0.86), Ice Days (r=0.82), Yearly Average Temperature (r= -0.99)
3. Variance Inflation Factor (VIF) kullanılarak her bir değişkenin diğer değişkenlerle ne derecede ilişkili olduğu saptanabilir.
    VIF > 10 : Yüksek multicollinearity
    VIF > 5 : Orta derecede multicollinearity
    VIF ≤ 5 : Kabul edilebilir düzeyde multicollinearity

## Multicollinearity Detection

In [39]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Sabit terim ekle
X = sm.add_constant(X)

# VIF hesaplama fonksiyonu
def calculate_vif(data):
    vif = pd.DataFrame()
    vif["feature"] = data.columns
    vif["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif

# VIF değerlerini hesapla
vif_data = calculate_vif(X)

print(vif_data)


                    feature            VIF
0                     const  373303.087440
1                      Year       8.688105
2             YearlyAvgTemp      24.103057
3             WinterAvgTemp     206.004043
4          WinterAvgMinTemp     241.826516
5             SummerAvgTemp      23.822031
6          SummerAvgMaxTemp      34.387410
7               SunlessDays       8.716656
8     DaysWithPrecipitation      15.524978
9               DaysWithFog       3.459170
10                  IceDays       7.840047
11                FrostDays      14.188423
12                 SnowDays      10.754705
13          HoursOfSunshine      12.696475
14              SummeryDays      14.251241
15             TropicalDays       6.589790
16                  DryDays       9.703272
17         RelativeHumidity       4.190716
18  QuantityOfPrecipitation       7.592877
19              Evaporation       4.181658


### Yorumlar:

const (Sabit Terim) : Bu göz ardı edilebilir.

## Yüksek Multicollineariy:

WinterAvgTemp (206.004043) ve WinterAvgMinTemp (241.826516):
Çok yüksek VIF değerleri gösteriyor. Bu değişkenler arasında ciddi bir multicollinearity var.
Muhtemelen bu değişkenler benzer bilgi taşıyor. Bunlardan birini çıkarabiliriz.

YearlyAvgTemp (24.103057), SummerAvgTemp (23.822031), SummerAvgMaxTemp (34.387410):
Yaz sıcaklıkları ve yıllık ortalama sıcaklıklar arasında multicollinearity görülüyor. Burada da bir seçim yapılabilir.
    
Diğer Değişkenler:
Örneğin, DaysWithFog (3.459170), RelativeHumidity (4.190716), ve Evaporation (4.181658) gibi değişkenler düşük VIF değerlerine sahip ve sorun yaratmıyor.

## Feature Selection

VIF değeri yüksek olan değişkenleri modelden çıkararak multicollinearity’yi azaltmayı deneyebiliriz.

Backward Elimination metodunu seçeceğiz. Modeldeki en az anlamlı değişkenleri adım adım çıkararak model performansı izlenebilir. 

### Backward Elimination

In [45]:
# Backward elimination uygulama
def backward_elimination(data, target, threshold=10):
    data_with_constant = add_constant(data)  # Sabit terim ekleniyor
    
    while True:
        # VIF hesapla
        vif = calculate_vif(data_with_constant)
        print("\nVIF Değerleri:")
        print(vif)
        
        # En yüksek VIF değerini bul
        max_vif = vif["VIF"].max()
        if max_vif > threshold:
            # VIF değeri en yüksek olan feature'ı seç
            max_vif_feature = vif.loc[vif["VIF"] == max_vif, "feature"].values[0]
            print(f"\nKaldırılan Değişken: {max_vif_feature} (VIF: {max_vif:.2f})")
            
            # Bu feature'ı veri setinden çıkar
            data_with_constant = data_with_constant.drop(columns=[max_vif_feature])
        else:
            break
    
    # Sabit terimi koruyarak nihai veri setini döndür
    return data_with_constant.drop(columns=['const'], errors='ignore')


In [47]:
# Backward Elimination kullanarak yeni değişken setini seç
X_reduced = backward_elimination(X, y)

# Yeni model oluştur ve eğit
model = sm.OLS(y, add_constant(X_reduced)).fit()

# Model sonuçlarını yazdır
print("\nFinal Model Sonuçları:")
print(model.summary())



VIF Değerleri:
                    feature            VIF
0                     const  373303.087440
1                      Year       8.688105
2             YearlyAvgTemp      24.103057
3             WinterAvgTemp     206.004043
4          WinterAvgMinTemp     241.826516
5             SummerAvgTemp      23.822031
6          SummerAvgMaxTemp      34.387410
7               SunlessDays       8.716656
8     DaysWithPrecipitation      15.524978
9               DaysWithFog       3.459170
10                  IceDays       7.840047
11                FrostDays      14.188423
12                 SnowDays      10.754705
13          HoursOfSunshine      12.696475
14              SummeryDays      14.251241
15             TropicalDays       6.589790
16                  DryDays       9.703272
17         RelativeHumidity       4.190716
18  QuantityOfPrecipitation       7.592877
19              Evaporation       4.181658

Kaldırılan Değişken: const (VIF: 373303.09)

VIF Değerleri:
                    

#### Yorum: Hala yüksek VIF değerine sahp değişkenler var. Bu değişkenleri veri setinden çıkarmak bir seçenek olabilir. Ancak bu kararı vermeden önce Final seti ile regresyon modelini yeniden çalıştırarak model performansını kontrol etmemiz gerek.  

Backward Elimination sonrası geriye kalan değişkenleri güncelliyoruz: Winter Average Min Temperature, Sunless Days, Days with Fog, Ice Days, Snow Days, Tropical Days

In [51]:
# HeatingDegreeDays bağımlı değişken
y = df["HeatingDegreeDays"]

# Backward elimination sonrası kalan değişkenler
X = df[['WinterAvgMinTemp', 'SunlessDays', 'DaysWithFog', 'IceDays','SnowDays', 'TropicalDays']]  

## Model Training 3.0: Eliminated Model

Bu modeli eğitim ve test setlerine random ayıracağız

In [54]:
from sklearn.model_selection import train_test_split

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split

# Eğitim ve test setlerine ayırma
X = df.drop("HeatingDegreeDays", axis=1)
y = df["HeatingDegreeDays"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model oluşturma ve eğitme
model = LinearRegression()
model.fit(X_train, y_train)

# Tahminleri oluşturma
y_train_pred = model.predict(X_train)  # Eğitim seti için tahminler
y_test_pred = model.predict(X_test)    # Test seti için tahminler

# Performans metriklerini hesaplama
mse_train = mean_squared_error(y_train, y_train_pred, squared=False)
mse_test = mean_squared_error(y_test, y_test_pred, squared=False)
mape_train = mean_absolute_percentage_error(y_train, y_train_pred)
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Sonuçları yazdırma
print(f"Training MSE: {mse_train}")
print(f"Test MSE: {mse_test}")
print(f"Training MAPE: {mape_train}")
print(f"Test MAPE: {mape_test}")
print(f"Training R-squared: {r2_train}")
print(f"Test R-squared: {r2_test}")



Training MSE: 15.77217169431068
Test MSE: 38.175703954140054
Training MAPE: 0.004261275005700654
Test MAPE: 0.010785274978758308
Training R-squared: 0.9969232426570068
Test R-squared: 0.9159785648513125


### Sonuç: Backward eliminetion öncesi ve sonrası modellerin performansı

In [70]:

data = {
    "Model": ["Original Set V.1", "Eliminated Set V.3"], 
    "Train MSE": [248.76, 15.77],    # Model 1 ve Model3 için Train MSE değerleri
    "Test MSE": [1457.38, 38.17],     # Model 1 ve Model3 için Test MSE değerleri
    "Train R²": [0.99, 0.99],         # Model 1 ve Model3 için Train R² değerleri
    "Test R²": [0.91, 0.91]           # Model 1 ve Model3 için Test R² değerleri
}

# Verileri bir DataFrame'e dönüştürme
comparison_df = pd.DataFrame(data)

# Tabloyu yazdırma
print(comparison_df)

                Model  Train MSE  Test MSE  Train R²  Test R²
0    Original Set V.1     248.76   1457.38      0.99     0.91
1  Eliminated Set V.3      15.77     38.17      0.99     0.91


### HDD için model eğitiminin sonu 